# Problema de transporte discreto

Dadas dos distribuciones de probabilidad discretas:

* $X$: con soporte $x_1<x_2<\ldots<x_m$ con probabilidades $P(X=x_i) = p_i$.
* $Y$: con soporte $y_1<y_2<\ldots<y_n$ con probabilidades $P(Y=y_j) = q_j$.

Queremos hallar una distribución conjunta $A=(a_{ij}) \geqslant 0$ tal que:

$$\min_{a_{ij}} \sum_{i,j} a_{ij} |x_i-y_j|$$

sujeto a:

$$\sum_j a_{ij} = p_i \quad \forall i=1,\ldots,m,$$

$$\sum_i a_{ij} = q_j \quad \forall j=1,\ldots,n.$$

In [11]:
## Para instalar los paquetes del Project.toml la primera vez
#using Pkg; Pkg.instantiate()

In [1]:
using JuMP, GLPK, Ipopt

#m=3
x=[1.0;2.0;3.0]
p=[0.2;0.4;0.4]

#n=2
y=[0;4.0]
q=[0.5;0.5]

#model = JuMP.Model(GLPK.Optimizer)
model = JuMP.Model(Ipopt.Optimizer)


m=length(x)
n=length(y)

#Calculo los pesos |xi-yj|
W=abs.(x*ones(1,n) - ones(m,1)*y')

@variable(model,A[1:m,1:n]>=0)

conp = @constraint(model, sum(A, dims=2).==p)
conq = @constraint(model, sum(A, dims=1).==q')

@objective(model,Min, sum( A.*W ))

model

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt
Names registered in the model: A

In [2]:
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        5
Total number of inequality co

In [3]:
objective_value(model)

1.3999999616644336

In [4]:
#Extraigo el valor optimo
value.(A)

3×2 Matrix{Float64}:
  0.2         -9.54545e-9
  0.3          0.1
 -9.54545e-9   0.4

In [5]:
#Extraigo los multiplicadores de p
lambda = dual.(conp)

3×1 Matrix{Float64}:
 1.7120871502758715e18
 1.7120871502758715e18
 1.7120871502758715e18

In [6]:
#Extraigo los multiplicadores de q
mu = dual.(conq)

1×2 Matrix{Float64}:
 -1.71209e18  -1.71209e18

In [7]:
W

3×2 Matrix{Float64}:
 1.0  3.0
 2.0  2.0
 3.0  1.0

In [8]:
lambda * ones(1,n) + ones(m,1)*mu

3×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0
 0.0  0.0

# Problema de transporte discreto - Cantidad de destinos variable

Dadas dos distribuciones de probabilidad discretas:

* $X$: con soporte $x_1<x_2<\ldots<x_m$ con probabilidades $P(X=x_i) = p_i$.
* $Y$: con soporte $y_1<y_2<\ldots<y_n$ con probabilidades $P(Y=y_j) = q_j$.

Queremos hallar una distribución conjunta $A=(a_{ij}) \geqslant 0$ tal que:

$$\min_{a_{ij}} \sum_{i,j} a_{ij} |x_i-y_j| + \epsilon \sum_{j} \phi(q_{j})$$

sujeto a:

$$\sum_j a_{ij} = p_i \quad \forall i=1,\ldots,m,$$

$$\sum_i a_{ij} = q_j \quad (VAR).$$

## Caso 1: 

Consideramos $\phi(x) = \frac{x^{2}}{2}$

In [9]:
using JuMP, GLPK, Ipopt
#revisar paquete distances.JL

#m=3
x=10*rand(10) #[1.0;2.0;3.0;4;5;6;7;8;9;10] #--> Trabajar con elementos de R^2
p=ones(10)

#n=2
y=[0;7;15]

#epsilon
eps = 0 #¿Qué pongo?

model = JuMP.Model(GLPK.Optimizer)
#model = JuMP.Model(Ipopt.Optimizer)


m=length(x)
n=length(y)

#Calculo los pesos |xi-yj|
W=abs.(x*ones(1,n) - ones(m,1)*y')

@variable(model,A[1:m,1:n]>=0)
@variable(model,q[1:n])

conp = @constraint(model, sum(A, dims=2).==p)
conq = @constraint(model, sum(A, dims=1).==q')

@objective(model,Min, sum( A.*W ))# + eps/2 * (q'*q) )

model

A JuMP Model
Minimization problem with:
Variables: 33
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 13 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 30 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: A, q

In [10]:
optimize!(model)

In [11]:
objective_value(model)

17.326311579433764

In [12]:
#Extraigo el valor optimo
value.(A)

10×3 Matrix{Float64}:
 1.0  0.0  0.0
 1.0  0.0  0.0
 1.0  0.0  0.0
 1.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  1.0  0.0
 1.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [13]:
value.(q)

3-element Vector{Float64}:
 7.0
 3.0
 0.0

In [14]:
#Extraigo los multiplicadores de p
lambda = dual.(conp)

10×1 Matrix{Float64}:
 1.0252439810761727
 2.2723018681825646
 1.931847834946323
 2.53611361313959
 1.0614259887028443
 2.136825543448058
 1.5251053758739792
 3.4181905278818085
 0.30246788069384634
 1.1167889654885725

In [15]:
#hacerlo en R^2,dentro de un cuadrado de 0,0 a 1,1
#plotear con el comando scatter los x, con diferentes colores los que tienen el 1 en la primer columna, los que tienen el 1 en la segunda col, etc.
#manera más sencilla con marker.z
#spliteo el X y corro la optimización, voy cambiando los cargadores
#

## Caso 2: Trabajando en $R^{2}$

Consideramos $\phi(x) = \frac{x^{2}}{2}$

In [16]:
#m=los que quiera
m = 5000
x = sortslices(rand(m,2), dims = 1);
p = ones(m);

#n=4
#y=[[1/3 1/3]; [1/3 2/3]; [2/3 1/3]; [2/3 2/3]];
#y=[[1/3 1/3]; [2/3 2/3]];
y = sortslices(rand(6,2), dims = 1);
n=size(y)[1];

In [17]:
using JuMP, GLPK, Ipopt
#revisar paquete distances.JL

#epsilon
eps = 0#0.001 #200

#model = JuMP.Model(GLPK.Optimizer)
model = JuMP.Model(Ipopt.Optimizer)

#Calculo los pesos |xi-yj|
w1 = (x[:,1]*ones(1,n) - ones(m,1)*y[:,1]').^2
w2 = (x[:,2]*ones(1,n) - ones(m,1)*y[:,2]').^2
W=(w1+w2).^(1/2)

@variable(model,A[1:m,1:n]>=0)
@variable(model,q[1:n])#<=833)

conp = @constraint(model, sum(A, dims=2).==p)
conq = @constraint(model, sum(A, dims=1).==q')

@objective(model,Min, sum( A.*W ) + eps/2 * (q'*q) )

model

A JuMP Model
Minimization problem with:
Variables: 30006
Objective function type: QuadExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 5006 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 30000 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt
Names registered in the model: A, q

In [18]:
optimize!(model);

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    60006
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    30006
                     variables with only lower bounds:    30000
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     5006
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.5899309e+02 5.00e+01 6.66e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [19]:
objective_value(model)

1065.600833955427

In [20]:
#Extraigo el valor optimo
value.(A)

5000×6 Matrix{Float64}:
 -2.89666e-9   1.0           1.18562e-8   …  -5.41865e-9   -6.54212e-9
  1.0          4.67098e-9    1.05503e-8      -2.44728e-10  -6.02224e-9
  1.0         -6.27476e-9   -5.57568e-9      -3.46556e-9   -7.31892e-9
  1.0          1.10872e-8    2.14903e-8       9.0495e-11   -5.83483e-9
  1.84817e-9   1.0           3.63691e-8      -4.01712e-9   -6.13629e-9
  1.64984e-8   0.999996      1.74302e-7   …  -2.02544e-9   -5.73138e-9
 -5.86686e-9   1.0           6.78668e-10     -6.76123e-9   -6.97576e-9
  1.0         -1.10522e-9    1.68017e-9      -9.23372e-10  -6.37398e-9
 -4.04168e-9   1.0           7.08125e-9      -5.8785e-9    -6.70084e-9
  3.83489e-9   1.0           4.96428e-8      -3.59032e-9   -6.0325e-9
  1.0         -7.42939e-10   2.23015e-9   …  -8.41594e-10  -6.33864e-9
  1.0         -5.47311e-9   -4.50497e-9      -2.68969e-9   -7.07187e-9
 -5.32009e-9   1.0           2.25384e-9      -6.47995e-9   -6.91809e-9
  ⋮                                       ⋱           

In [21]:
value.(q)

6-element Vector{Float64}:
  492.0000395414625
  480.99996223676965
  684.9999311034252
  772.0002385435363
 1001.9999972229407
 1567.999831351871

In [22]:
#Extraigo los multiplicadores de p
lambda = dual.(conp);

In [23]:
using VoronoiCells
using GeometryBasics
using Plots
using Random

#Redondea los valores de A para que sean 0 o 1 (Probar)
Ar = round.(value.(A));

gr()
Plots.GRBackend()

#Separa los puntos por destino
X = [x[findall(x->x==1, Ar[:,i]),1] for i in 1:n]
Y = [x[findall(x->x==1, Ar[:,i]),2] for i in 1:n]
points = [y[:,1], y[:,2]]

#Conjuntos de Voronoi
rect = Rectangle(Point2(0, 0), Point2(1, 1))
tess = voronoicells(points, rect);

scatter(X, Y, aspect_ratio=:equal) #Grafica los origenes, con color por destino
scatter!(points, markersize = 10, label = "Destinos") #Plotea los destinos
annotate!([(points[n][1] + 0.02, points[n][2] + 0.03, Plots.text(n)) for n in 1:10])
plot!(tess, legend = :topleft)

LoadError: MethodError: no method matching getx(::Vector{Float64})
[0mClosest candidates are:
[0m  getx([91m::Point2[39m) at ~/.julia/packages/VoronoiCells/RtEBt/src/Points.jl:37
[0m  getx([91m::GeometricalPredicates.Point2D[39m) at ~/.julia/packages/GeometricalPredicates/1dBKW/src/GeometricalPredicates.jl:88
[0m  getx([91m::GeometricalPredicates.Point3D[39m) at ~/.julia/packages/GeometricalPredicates/1dBKW/src/GeometricalPredicates.jl:99
[0m  ...